In [3]:
import random
import torch
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.metrics import mean_squared_error, accuracy_score, mean_absolute_error, r2_score, mean_squared_error
from sklearn.model_selection import train_test_split,GridSearchCV
        

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [4]:
def fix_random(seed: int) -> None:
    """Fix all the possible sources of randomness.

    Args:
        seed: the seed to use.
    """
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True  # slower

In [5]:
SEED = 1038893

fix_random(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))

root = "../../data/ml-25m"

Device: cpu


## Data Acquisition

In [6]:
def get_data_from_csv(file: str, nrows=None):
    if nrows:
        df = pd.read_csv(f"{root}/{file}", nrows=nrows)
    else:
        df = pd.read_csv(f"{root}/{file}")
    print(f"Loaded ml-25m data: {root}/{file}")

    return df

def get_ratings_from_csv(nrows=None):
    path = "ratings.csv"

    data = get_data_from_csv(path, nrows)
    # data = get_data_from_csv(path)

    data.drop("timestamp", axis=1, inplace=True)
    # todo: drop user id
    #data.drop("userId", axis=1, inplace=True)

    return data

def get_tag_relevances_from_csv(nrows=None):
    path = "genome-scores.csv"

    return get_data_from_csv(path, nrows)

In [7]:
genome_scores_path = "genome-scores.csv"
ratings_path = "ratings.csv"

In [8]:
ratings = get_ratings_from_csv()
movies = get_data_from_csv("movies.csv")
print(ratings)

Loaded ml-25m data: ../../data/ml-25m/ratings.csv
Loaded ml-25m data: ../../data/ml-25m/movies.csv
          userId  movieId  rating
0              1      296     5.0
1              1      306     3.5
2              1      307     5.0
3              1      665     5.0
4              1      899     3.5
...          ...      ...     ...
25000090  162541    50872     4.5
25000091  162541    55768     2.5
25000092  162541    56176     2.0
25000093  162541    58559     4.0
25000094  162541    63876     5.0

[25000095 rows x 3 columns]


In [9]:
genome_scores = get_tag_relevances_from_csv()
print(genome_scores)

Loaded ml-25m data: ../../data/ml-25m/genome-scores.csv
          movieId  tagId  relevance
0               1      1    0.02875
1               1      2    0.02375
2               1      3    0.06250
3               1      4    0.07575
4               1      5    0.14075
...           ...    ...        ...
15584443   206499   1124    0.11000
15584444   206499   1125    0.04850
15584445   206499   1126    0.01325
15584446   206499   1127    0.14025
15584447   206499   1128    0.03350

[15584448 rows x 3 columns]


In [10]:
# films = get_data_from_csv(f"{root}/{ratings}")[]


# DATA VISUALIZATION


In [11]:
            # # Fill in missing values with zeros
            # X.fillna(0, inplace=True)


In [12]:
def addColumnOperation(ratings,X):
     # Compute the mean rating for each user
     count_rating = ratings.groupby('movieId', as_index=False)['rating'].count()
     std= ratings.groupby('movieId', as_index=False)['rating'].std()
     std.fillna(0, inplace=True)
     min_ratings= ratings.groupby('movieId', as_index=False)['rating'].min()
     max_ratings= ratings.groupby('movieId', as_index=False)['rating'].max()
     median= ratings.groupby('movieId', as_index=False)['rating'].median()
     operation = pd.DataFrame({'movieId':count_rating['movieId'],'count_rating': count_rating['rating'], 'std': std['rating'], 'min': min_ratings['rating'], 'max': max_ratings['rating'], 'median': median['rating']}) 
     X = pd.merge(X, operation, on='movieId')
     X.drop("movieId", axis=1, inplace=True)
     return X

# FARE TEST CON AVG, STD_DEV

In [13]:
class SVR:
    def __init__(self, ratings, relevance,movies, seed=1038893):
        from sklearn.svm import SVR
        from sklearn.pipeline import Pipeline
        from sklearn.preprocessing import StandardScaler
        from sklearn.decomposition import PCA

        # Pivot the relevance DataFrame to create a matrix of tag relevance scores for each movie
        relevance_matrix = relevance.pivot_table(index='movieId', columns='tagId', values='relevance', fill_value=0)

        # Merge the ratings and relevance data
        mean_ratings = ratings.groupby('movieId', as_index=False)['rating'].mean()
        X = mean_ratings.merge(relevance_matrix, on='movieId')
        X.columns = X.columns.astype(str)
        
        ratings = None  

        # mescolare le righe del DataFrame
        X = X.sample(frac=1,random_state=seed).reset_index(drop=True)
       
        
        X.drop("movieId", axis=1, inplace=True)
        self.y = X['rating']
        self.X = X.drop('rating', axis=1)
        

        # Split the data into training and testing sets
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size=0.2, random_state=seed)
        #print(self.X_train.shape, self.X_test.shape, self.y_train.shape, self.y_test.shape)

        # Initialize the svr model
        #self.svr = SVR(C=1, epsilon=0.1,kernel='linear')

        
        self.pipelineScaled = Pipeline([('Standard Scaler',StandardScaler()),('pca',PCA(n_components=0.95)),('Linear Regression',SVR(C=1, epsilon=0.1,kernel='linear'))],verbose=True)
        self.pipeline = Pipeline([('pca',PCA(n_components=0.95)),('Linear Regression',SVR(C=1, epsilon=0.1,kernel='linear'))],verbose=True)
        self.svr = Pipeline([SVR(C=1, epsilon=0.1,kernel='linear')],verbose=True)

        self.params = {
            'kernel': ['linear','poly', 'rbf', 'sigmoid'],
            'C': [0.01, 0.1, 1, 10, 100],
            'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1, 10],
            'epsilon': [0.01, 0.1, 1, 10]
        }

        #In caso il Kernel sia 'Linear' il modello sará analogo a usare LinearSVR

    # Training
    def train(self):
        for model in [self.pipelineScaled,self.pipeline,self.svr]:
            model.fit(self.X_train, self.y_train)
    
    def tuning(self):
        from sklearn.model_selection import RandomizedSearchCV
        grid = RandomizedSearchCV(estimator = self.svr, param_distributions = self.params, cv = 5, n_jobs = -1, verbose = 2)
        grid.fit(self.X_train, self.y_train)
        print(grid.best_params_)
        self.svr = grid.best_estimator_
        

    def test(self):
        for model in [self.pipelineScaled,self.pipeline,self.svr]:
            # Predict ratings for the test data
            y_pred = model.predict(self.X_test)

            # Compute the mean squared error
            mse = mean_squared_error(self.y_test, y_pred)
            # Compute R^2
            r2 = r2_score(self.y_test, y_pred)
            # Compute the mean absolute error
            mae = mean_absolute_error(self.y_test, y_pred)

            print(f"Mean squared error: {mse}")
            print(f"R^2: {r2}")
            print(f"Mean absolute error: {mae}")
            print("=====================================")

In [14]:
bs = SVR(ratings, genome_scores,movies, SEED)

(13816, 1128)


C:\Users\simon\AppData\Local\Temp\ipykernel_17432\2152857862.py:13: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  m = pd.get_dummies(movies_genres.apply(pd.Series).stack()).sum(level=0)


In [15]:
#bs.tuning()

In [16]:
bs.train()

[Pipeline] ... (step 1 of 3) Processing Standard Scaler, total=   0.8s
[Pipeline] ............... (step 2 of 3) Processing pca, total=  10.0s


In [ ]:
bs.test()

Mean squared error: 0.0060253705578357086
R^2: 0.973824480969006
Mean absolute error: 0.06057859549894352
